In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import pandas as pd
import numpy as np

import  matplotlib.pyplot as plt
import pickle

from chemistry import Molecule
from util import score
from pprint import pprint


In [3]:
with open('../data/molecules_enh.pickle', 'rb') as f:
    molecules = pickle.load(f)

In [4]:
structures = pd.read_feather('../data/structures_enh.feather')
molecules_df = pd.read_feather('../data/molecules.feather')
labelled = pd.read_feather('../data/train.feather')
unlabelled = pd.read_feather('../data/test.feather')

In [5]:
labelled_enh = labelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
#labelled_enh.head(32)

In [6]:
unlabelled_enh = unlabelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
#unlabelled_enh.head(32)

In [11]:
from models import SKModel, partition_data

def fn(data_df, n):
    global numeric_df
    data_df = labelled[labelled.type == '1JHC'].head(n)
    #data_df = labelled_enh.head(16)
    train_df, valid_df, test_df = partition_data(data_df, train_frac=1)
    train_df = train_df#.copy()

    model = SKModel(dict(molecules=molecules,
                         structures=structures), {})
    input_df, numeric_df, output_df = model.setup_data(train_df, train_df)

%lprun -f SKModel.make_input fn(labelled_enh, 10)

Timer unit: 1e-06 s

Total time: 4.1348 s
File: /home/gautham/work/kaggle/scalar_coupling/notebooks/models.py
Function: make_input at line 172

Line #      Hits         Time  Per Hit   % Time  Line Contents
   172                                               def make_input(self, input_df):
   173         1    1575517.0 1575517.0     38.1          df = self.make_dist(input_df)
   174         1        167.0    167.0      0.0          df['type'] = input_df['type']
   175                                           
   176         2          8.0      4.0      0.0          for t in self.coupling_types:
   177         1       1743.0   1743.0      0.0              df[f'coupling_{t}'] = (df.type == t).astype('int8')
   178                                           
   179         1     819144.0 819144.0     19.8          self.make_atom_columns(df, df.atom_0, df.atom_index_0, 'atom0')
   180         1     833766.0 833766.0     20.2          self.make_atom_columns(df, df.atom_1, df.atom_index_1, 

In [ ]:
list(zip(numeric_df.columns,numeric_df.loc[0]))